<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

# Read, Cache and Prefetch

In [2]:
batch_size = 32
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

AUTOTUNE = tf.data.AUTOTUNE

train = train.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
val = val.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)
test = test.cache().batch(batch_size).prefetch(buffer_size=AUTOTUNE)

# Augmentation

In [3]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    # tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

# Visualisation

In [4]:
def summary_conv_and_last_dense(model):
    lines = []
    total_params = 0
    
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
            lines.append((layer.name, type(layer).__name__, layer.output.shape, layer.count_params()))
            total_params += layer.count_params()

    if lines:
        text = "----------------------------------------------------------------"
        text += f"\n{'Layer':<8}{'Output Shape':<20}{'Param #':>10}"
        text += "\n================================================================"
        for name, ltype, shape, params in lines:
            text += f"\n{ltype:<8}{str(shape):<20}{params:>10}"
        text += "\n================================================================"
        text += f"\nTotal params: {total_params}"
        print(text)
    else:
        print("No Conv2D or Dense layers found in the model.")

# Diff Evo and Func

In [10]:
def DE(pop_size, iterations, function, limits, ints):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    if ints:
        for it in ints:
            population[:, it] = np.round(population[:, it])

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    # function calls = pop_size + iterations * pop_size
    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            if ints:
                for it in ints:
                    mutant_vector[it] = np.round(mutant_vector[it])
            mutant_vector = np.clip(mutant_vector, x_low, x_high)
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    X = X.astype(int)
    amount_of_layer_1 = X[0]
    amount_of_layer_2 = X[1]
    amount_of_layer_3 = X[2]
    amount_of_one_dence = X[3]
    amount_of_one_filters = X[4]

    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters * 2), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(int(amount_of_one_filters * 4), (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(int(amount_of_one_filters * 8), activation='relu'))
    model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    # model.summary()
    # print(model.count_params())
    summary_conv_and_last_dense(model)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(train, batch_size=batch_size, epochs=100, validation_data=(val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    evaluation = model.evaluate(test)
    # print(model.metrics_names)
    # print(evaluation)
    loss, acc, top2  = evaluation
    # return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]
    return (1.0 - acc) + loss

func_limits = [[1, 4], [0, 4], [0, 4], [1, 4], [2, 6]]


# Use DE

In [11]:
best_f, best_d = DE(4, 4, func, func_limits, ints=[0, 1, 2, 3, 4])
best_f, best_d

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 4)          112
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 16, 16, 8)          296
Dense   (None, 32)                1056
Dense   (None, 32)                1056
Dense   (None, 32)                1056
Dense   (None, 32)                1056
Dense   (None, 10)                 330
Total params: 5110
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - Top2: 0.2510 - accuracy: 0.1280 - loss: 2.3477 - val_Top2: 0.4015 - val_accuracy: 0.2065 - val_loss: 2.0880
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - Top2: 0.3684 - accuracy: 0.1928 - loss: 2.1253 - val_Top2: 0.4093 - val_accuracy: 0.2170 - val_loss: 2.0544
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - Top2: 0.3951 - accuracy: 0.2057 - loss: 2.0759 - val_Top2: 0.4531 - val_accuracy: 0.2400 - val_loss: 1.9813
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - Top2: 0.4

Processing:   0%|                                        | 0/4 [00:00<?, ?step/s]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 4)          112
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 8, 8, 16)           592
Conv2D  (None, 8, 8, 16)          2320
Conv2D  (None, 8, 8, 16)          2320
Conv2D  (None, 8, 8, 16)          2320
Dense   (None, 32)                2080
Dense   (None, 10)                 330
Total params: 10518
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - Top2: 0.2205 - accuracy: 0.1127 - loss: 2.7523 - val_Top2: 0.3557 - val_accuracy: 0.1911 - val_loss: 2.1452
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - Top2: 0.3104 - accuracy: 0.1612 - loss: 2.1878 - val_Top2: 0.3807 - val_accuracy: 0.1847 - val_loss: 2.0382
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - Top2: 0.3421 - accuracy: 0.1752 - loss: 2.1251 - val_Top2: 0.3901 - val_accuracy: 0.1989 - va

Processing:  25%|██████████                              | 1/4 [27:04<1:21:14, 1624.71s/step]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 4)          112
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 32, 32, 4)          148
Conv2D  (None, 8, 8, 16)           592
Conv2D  (None, 8, 8, 16)          2320
Conv2D  (None, 8, 8, 16)          2320
Conv2D  (None, 8, 8, 16)          2320
Dense   (None, 32)                2080
Dense   (None, 10)                 330
Total params: 10518
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - Top2: 0.2128 - accuracy: 0.1089 - loss: 2.8114 - val_Top2: 0.2916 - val_accuracy: 0.1476 - val_loss: 2.2396
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - Top2: 0.2823 - accuracy: 0.1461 - loss: 2.2415 - val_Top2: 0.3485 - val_accuracy: 0.1743 - val_loss: 2.2486
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - Top2: 0.3683 - accuracy: 0.1898 - loss: 2.1299 - val_Top2: 0.4451 - val_accuracy: 0.2310 - va

...

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 6)          168
Conv2D  (None, 32, 32, 6)          330
Conv2D  (None, 32, 32, 6)          330
Conv2D  (None, 32, 32, 6)          330
Conv2D  (None, 16, 16, 12)         660
Conv2D  (None, 16, 16, 12)        1308
Conv2D  (None, 16, 16, 12)        1308
Conv2D  (None, 16, 16, 12)        1308
Conv2D  (None, 8, 8, 24)          2616
Conv2D  (None, 8, 8, 24)          5208
Conv2D  (None, 8, 8, 24)          5208
Conv2D  (None, 8, 8, 24)          5208
Dense   (None, 48)                4656
Dense   (None, 48)                2352
Dense   (None, 48)                2352
Dense   (None, 48)                2352
Dense   (None, 10)                 490
Total params: 36184
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 25ms/step - Top2: 0.2508 - accuracy: 0.1258 - loss: 2.2940 - val_Top2: 0.3788 - val_accuracy: 0.2045 - val_loss: 2.0317
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 31s

Processing:  75%|██████████████████████████████          | 3/4 [1:44:44<37:38, 2258.10s/step]

----------------------------------------------------------------
Layer   Output Shape           Param #
Conv2D  (None, 32, 32, 5)          140
Conv2D  (None, 32, 32, 5)          230
Conv2D  (None, 8, 8, 20)           920
Conv2D  (None, 8, 8, 20)          3620
Conv2D  (None, 8, 8, 20)          3620
Conv2D  (None, 8, 8, 20)          3620
Dense   (None, 40)                3240
Dense   (None, 10)                 410
Total params: 15800
Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - Top2: 0.2334 - accuracy: 0.1212 - loss: 2.6907 - val_Top2: 0.4338 - val_accuracy: 0.2183 - val_loss: 2.0707
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - Top2: 0.3640 - accuracy: 0.1869 - loss: 2.1477 - val_Top2: 0.4698 - val_accuracy: 0.2549 - val_loss: 1.9539
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - Top2: 0.4166 - accuracy: 0.2162 - loss: 2.0575 - val_Top2: 0.4819 - val_accuracy: 0.2537 - val_loss: 1.9083
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - Top2

Processing: 100%|████████████████████████████████████████| 4/4 [2:22:40<00:00, 2140.00s/step]


(np.float64(1.8723938465118408), array([4., 4., 4., 4., 6.]))

# Bests apart

In [ ]:
best_f, best_d

# Plains

## Func

In [ ]:
def func(X):
    amount_of_layer_1 = 1
    amount_of_layer_2 = 1
    amount_of_layer_3 = 1
    amount_of_one_dence = 1
    amount_of_one_filters = int(round(X[0]))
    Dropout = X[1]
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    for _ in range(amount_of_layer_2):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))
    model_layers.append(layers.Dropout(0.3))

    for _ in range(amount_of_layer_3):
        model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(4, 4))
    model_layers.append(layers.Dropout(0.4))

    model_layers.append(layers.Flatten())
    for _ in range(amount_of_one_dence):
        model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    model_layers.append(layers.Dropout(Dropout))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_loss'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[16, 128], [0.01, 0.99]]

## Configuration

In [ ]:
dots = 8
x1_lim = func_limits[0]
x1 = np.linspace(x1_lim[0], x1_lim[1], dots)
x2_lim = func_limits[1]
x2 = np.linspace(x2_lim[0], x2_lim[1], dots)
space = np.meshgrid(x1, x2)

In [ ]:
fitness = np.array([[func(np.array([i, j]))] for j in x2] for i in x1)

## Plotting

## Thrash

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)